In [201]:
import pandas as pd

historical_weather = pd.read_csv('historical_weather.csv')
sample_submission = pd.read_csv('sample_submission.csv')
submission_key = pd.read_csv('submission_key.csv')

In [202]:
historical_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182338 entries, 0 to 182337
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   city_id             182338 non-null  object 
 1   date                182338 non-null  object 
 2   avg_temp_c          181114 non-null  float64
 3   min_temp_c          176452 non-null  float64
 4   max_temp_c          174845 non-null  float64
 5   precipitation_mm    112594 non-null  float64
 6   snow_depth_mm       12238 non-null   float64
 7   avg_wind_dir_deg    146944 non-null  float64
 8   avg_wind_speed_kmh  159866 non-null  float64
dtypes: float64(7), object(2)
memory usage: 12.5+ MB


In [203]:
# Check for missing values
print(historical_weather.isnull().sum())

city_id                    0
date                       0
avg_temp_c              1224
min_temp_c              5886
max_temp_c              7493
precipitation_mm       69744
snow_depth_mm         170100
avg_wind_dir_deg       35394
avg_wind_speed_kmh     22472
dtype: int64


In [204]:
# Threshold for dropping columns
threshold = 0.5  # If more than 50% values are missing

# Drop columns with excessive missing values
historical_weather = historical_weather.loc[:, historical_weather.isnull().mean() < threshold]

In [207]:
historical_weather.head()

,city_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,avg_wind_dir_deg,avg_wind_speed_kmh
0,C001,2014-01-01,6.6,-1.4,11.6,NaN,168.0,6.2
1,C001,2014-01-02,9.3,6.3,13.3,NaN,155.0,10.0
2,C001,2014-01-03,7.6,1.9,14.0,NaN,NaN,5.8
3,C001,2014-01-04,7.6,3.9,13.3,NaN,291.0,11.3
4,C001,2014-01-05,8.6,0.5,16.9,NaN,NaN,5.0


In [208]:
# Check for missing values
print(historical_weather.isnull().sum())

# Fill missing values using interpolation
#If data set has lare gap, sometimes this will not be working properly.
historical_weather.interpolate(method='linear', inplace=True)

historical_weather.head()

city_id                   0
date                      0
avg_temp_c             1224
min_temp_c             5886
max_temp_c             7493
precipitation_mm      69744
avg_wind_dir_deg      35394
avg_wind_speed_kmh    22472
dtype: int64


,city_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,avg_wind_dir_deg,avg_wind_speed_kmh
0,C001,2014-01-01,6.6,-1.4,11.6,NaN,168.0,6.2
1,C001,2014-01-02,9.3,6.3,13.3,NaN,155.0,10.0
2,C001,2014-01-03,7.6,1.9,14.0,NaN,223.0,5.8
3,C001,2014-01-04,7.6,3.9,13.3,NaN,291.0,11.3
4,C001,2014-01-05,8.6,0.5,16.9,NaN,223.5,5.0


In [209]:
historical_weather.head()

,city_id,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,avg_wind_dir_deg,avg_wind_speed_kmh
0,C001,2014-01-01,6.6,-1.4,11.6,NaN,168.0,6.2
1,C001,2014-01-02,9.3,6.3,13.3,NaN,155.0,10.0
2,C001,2014-01-03,7.6,1.9,14.0,NaN,223.0,5.8
3,C001,2014-01-04,7.6,3.9,13.3,NaN,291.0,11.3
4,C001,2014-01-05,8.6,0.5,16.9,NaN,223.5,5.0


In [210]:
historical_weather['date'] = pd.to_datetime(historical_weather['date'])

In [211]:
#The values of precipitation_mm has large gap. So we use this mehod fill large gap null values.
from sklearn.impute import KNNImputer

# Extract the precipitation_mm column
precipitation_mm = historical_weather[['precipitation_mm']]

imputer = KNNImputer(n_neighbors=5)
precipitation_mm_imputed = pd.DataFrame(imputer.fit_transform(precipitation_mm), columns=['precipitation_mm'])

# Replace the original precipitation_mm column with the imputed values
historical_weather['precipitation_mm'] = precipitation_mm_imputed['precipitation_mm']

In [212]:
# Check for any remaining missing values
print(historical_weather.isnull().sum())

# Display the first few rows of the DataFrame to verify the changes
print(historical_weather.head())

city_id               0
date                  0
avg_temp_c            0
min_temp_c            0
max_temp_c            0
precipitation_mm      0
avg_wind_dir_deg      0
avg_wind_speed_kmh    0
dtype: int64
  city_id       date  avg_temp_c  min_temp_c  max_temp_c  precipitation_mm  \
0    C001 2014-01-01         6.6        -1.4        11.6          3.959916   
1    C001 2014-01-02         9.3         6.3        13.3          3.959916   
2    C001 2014-01-03         7.6         1.9        14.0          3.959916   
3    C001 2014-01-04         7.6         3.9        13.3          3.959916   
4    C001 2014-01-05         8.6         0.5        16.9          3.959916   

   avg_wind_dir_deg  avg_wind_speed_kmh  
0             168.0                 6.2  
1             155.0                10.0  
2             223.0                 5.8  
3             291.0                11.3  
4             223.5                 5.0  


In [213]:
# One-hot encode categorical variables
data = pd.get_dummies(historical_weather, columns=['city_id'])

In [214]:
# Convert date to datetime format
data['date'] = pd.to_datetime(data['date'])

# Extract year, month, and day
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

In [215]:
data.head()

,date,avg_temp_c,min_temp_c,max_temp_c,precipitation_mm,avg_wind_dir_deg,avg_wind_speed_kmh,city_id_C001,city_id_C002,city_id_C003,...,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,year,month,day
0,2014-01-01,6.6,-1.4,11.6,3.959916,168.0,6.2,1,0,0,...,0,0,0,0,0,0,0,2014,1,1
1,2014-01-02,9.3,6.3,13.3,3.959916,155.0,10.0,1,0,0,...,0,0,0,0,0,0,0,2014,1,2
2,2014-01-03,7.6,1.9,14.0,3.959916,223.0,5.8,1,0,0,...,0,0,0,0,0,0,0,2014,1,3
3,2014-01-04,7.6,3.9,13.3,3.959916,291.0,11.3,1,0,0,...,0,0,0,0,0,0,0,2014,1,4
4,2014-01-05,8.6,0.5,16.9,3.959916,223.5,5.0,1,0,0,...,0,0,0,0,0,0,0,2014,1,5


In [216]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error,r2_score

In [241]:
features_avgtemp2 = [col for col in data.columns if col not in ['date','avg_temp_c','precipitation_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh']]
print(features_avgtemp2)

['min_temp_c', 'max_temp_c', 'city_id_C001', 'city_id_C002', 'city_id_C003', 'city_id_C004', 'city_id_C005', 'city_id_C007', 'city_id_C008', 'city_id_C009', 'city_id_C010', 'city_id_C011', 'city_id_C012', 'city_id_C013', 'city_id_C014', 'city_id_C015', 'city_id_C016', 'city_id_C017', 'city_id_C018', 'city_id_C020', 'city_id_C022', 'city_id_C023', 'city_id_C024', 'city_id_C025', 'city_id_C027', 'city_id_C028', 'city_id_C029', 'city_id_C030', 'city_id_C031', 'city_id_C033', 'city_id_C034', 'city_id_C035', 'city_id_C036', 'city_id_C037', 'city_id_C038', 'city_id_C039', 'city_id_C040', 'city_id_C042', 'city_id_C043', 'city_id_C044', 'city_id_C045', 'city_id_C046', 'city_id_C047', 'city_id_C048', 'city_id_C049', 'city_id_C051', 'city_id_C053', 'city_id_C054', 'city_id_C055', 'city_id_C056', 'city_id_C057', 'city_id_C058', 'city_id_C059', 'city_id_C061', 'city_id_C062', 'city_id_C064', 'city_id_C065', 'city_id_C066', 'city_id_C067', 'city_id_C068', 'city_id_C069', 'city_id_C070', 'city_id_C0

In [218]:
features_minmax_temp = [col for col in data.columns if col not in ['date','avg_temp_c', 'min_temp_c', 'max_temp_c','precipitation_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh']]

In [219]:
features_avgtemp = [col for col in data.columns if col not in ['date','avg_temp_c','precipitation_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh']]
features_minmax_temp = [col for col in data.columns if col not in ['date','avg_temp_c', 'min_temp_c', 'max_temp_c','precipitation_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh']]
features_all = [col for col in data.columns if col not in ['date', 'avg_temp_c', 'precipitation_mm', 'avg_wind_dir_deg', 'avg_wind_speed_kmh']]

X = data[features_all]
y_dummy = data['avg_temp_c']  # Using a dummy target for splitting
X_train, X_test, y_train_dummy, y_test_dummy = train_test_split(X, y_dummy, test_size=0.2, random_state=42)

# Get indices for training and testing sets
train_indices = X_train.index.tolist()
test_indices = X_test.index.tolist()

# Split data for each target variable using the same indices
def get_train_test_split(data, features, train_indices, test_indices, target):
    X = data[features]
    y = data[target]
    X_train = X.loc[train_indices]
    X_test = X.loc[test_indices]
    y_train = y.loc[train_indices]
    y_test = y.loc[test_indices]
    return X_train, X_test, y_train, y_test

# avg_temp_c
X_train_avg, X_test_avg, y_train_avg, y_test_avg = get_train_test_split(data, features_all, train_indices, test_indices, 'avg_temp_c')
# max_temp_c
X_train_max, X_test_max, y_train_max, y_test_max = get_train_test_split(data, features_minmax_temp, train_indices, test_indices, 'max_temp_c')
# min_temp_c
X_train_min, X_test_min, y_train_min, y_test_min = get_train_test_split(data, features_minmax_temp, train_indices, test_indices, 'min_temp_c')


In [220]:
# Train model
model_avg = RandomForestRegressor(n_estimators=100, random_state=42)
model_avg.fit(X_train_avg, y_train_avg)

RandomForestRegressor(random_state=42)

In [221]:
# Train model
model_min = RandomForestRegressor(n_estimators=100, random_state=42)
model_min.fit(X_train_min, y_train_min)

RandomForestRegressor(random_state=42)

In [222]:
# Train model
model_max = RandomForestRegressor(n_estimators=100, random_state=42)
model_max.fit(X_train_max, y_train_max)

RandomForestRegressor(random_state=42)

In [223]:
# Evaluate model
y_pred = model_avg.predict(X_test_avg)
mse = mean_squared_error(y_test_avg, y_pred)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test_avg, y_pred)
print(f'R-squared: {r2}')

Mean Squared Error: 0.9012097950830019
R-squared: 0.9908805349044751


In [224]:
submission_key = pd.read_csv('submission_key.csv')
submission_key.head()

,submission_ID,city_id,date
0,1,C001,2019-01-01
1,2,C001,2019-01-02
2,3,C001,2019-01-03
3,4,C001,2019-01-04
4,5,C001,2019-01-05


In [225]:
submission_key['date'] = pd.to_datetime(submission_key['date'])
submission_key['year'] = submission_key['date'].dt.year
submission_key['month'] = submission_key['date'].dt.month
submission_key['day'] = submission_key['date'].dt.day

In [226]:
submission_data = pd.get_dummies(submission_key, columns=['city_id'])

In [227]:
submission_data_new = submission_data.reindex(columns=features, fill_value=0)

In [228]:
submission_data.head()

,submission_ID,date,year,month,day,city_id_C001,city_id_C002,city_id_C003,city_id_C004,city_id_C005,...,city_id_C103,city_id_C104,city_id_C105,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112
0,1,2019-01-01,2019,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2019-01-02,2019,1,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,2019-01-03,2019,1,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,2019-01-04,2019,1,4,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,2019-01-05,2019,1,5,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [229]:
cols = list(submission_data.columns)
cols_to_move = ['year', 'month', 'day']
for col in cols_to_move:
    cols.remove(col)
cols += cols_to_move

submission_data = submission_data[cols]

In [230]:
submission_data.head()

,submission_ID,date,city_id_C001,city_id_C002,city_id_C003,city_id_C004,city_id_C005,city_id_C007,city_id_C008,city_id_C009,...,city_id_C106,city_id_C107,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,year,month,day
0,1,2019-01-01,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,1
1,2,2019-01-02,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,2
2,3,2019-01-03,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,3
3,4,2019-01-04,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,4
4,5,2019-01-05,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2019,1,5


In [231]:
features = [col for col in submission_data.columns if col not in ['submission_ID', 'date']]
print(features)

['city_id_C001', 'city_id_C002', 'city_id_C003', 'city_id_C004', 'city_id_C005', 'city_id_C007', 'city_id_C008', 'city_id_C009', 'city_id_C010', 'city_id_C011', 'city_id_C012', 'city_id_C013', 'city_id_C014', 'city_id_C015', 'city_id_C016', 'city_id_C017', 'city_id_C018', 'city_id_C020', 'city_id_C022', 'city_id_C023', 'city_id_C024', 'city_id_C025', 'city_id_C027', 'city_id_C028', 'city_id_C029', 'city_id_C030', 'city_id_C031', 'city_id_C033', 'city_id_C034', 'city_id_C035', 'city_id_C036', 'city_id_C037', 'city_id_C038', 'city_id_C039', 'city_id_C040', 'city_id_C042', 'city_id_C043', 'city_id_C044', 'city_id_C045', 'city_id_C046', 'city_id_C047', 'city_id_C048', 'city_id_C049', 'city_id_C051', 'city_id_C053', 'city_id_C054', 'city_id_C055', 'city_id_C056', 'city_id_C057', 'city_id_C058', 'city_id_C059', 'city_id_C061', 'city_id_C062', 'city_id_C064', 'city_id_C065', 'city_id_C066', 'city_id_C067', 'city_id_C068', 'city_id_C069', 'city_id_C070', 'city_id_C071', 'city_id_C072', 'city_i

In [232]:
X_submission = submission_data[features]
submission_data = submission_data.copy()
submission_data['min_temp_c'] = model_min.predict(X_submission)

In [233]:
X_submission = submission_data[features]
submission_data = submission_data.copy()
submission_data['max_temp_c'] = model_max.predict(X_submission)

In [234]:
submission_data.head()

,submission_ID,date,city_id_C001,city_id_C002,city_id_C003,city_id_C004,city_id_C005,city_id_C007,city_id_C008,city_id_C009,...,city_id_C108,city_id_C109,city_id_C110,city_id_C111,city_id_C112,year,month,day,min_temp_c,max_temp_c
0,1,2019-01-01,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2019,1,1,1.98,16.910
1,2,2019-01-02,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2019,1,2,1.46,18.420
2,3,2019-01-03,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2019,1,3,3.02,17.310
3,4,2019-01-04,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2019,1,4,6.93,16.279
4,5,2019-01-05,1,0,0,0,0,0,0,0,...,0,0,0,0,0,2019,1,5,6.57,16.460


In [238]:
cols = list(submission_data.columns)
cols_to_move = ['year', 'month', 'day']
for col in cols_to_move:
    cols.remove(col)
cols += cols_to_move

submission_data = submission_data[cols]

In [239]:
features_new = [col for col in submission_data.columns if col not in ['submission_ID', 'date']]
print(features_new)

['city_id_C001', 'city_id_C002', 'city_id_C003', 'city_id_C004', 'city_id_C005', 'city_id_C007', 'city_id_C008', 'city_id_C009', 'city_id_C010', 'city_id_C011', 'city_id_C012', 'city_id_C013', 'city_id_C014', 'city_id_C015', 'city_id_C016', 'city_id_C017', 'city_id_C018', 'city_id_C020', 'city_id_C022', 'city_id_C023', 'city_id_C024', 'city_id_C025', 'city_id_C027', 'city_id_C028', 'city_id_C029', 'city_id_C030', 'city_id_C031', 'city_id_C033', 'city_id_C034', 'city_id_C035', 'city_id_C036', 'city_id_C037', 'city_id_C038', 'city_id_C039', 'city_id_C040', 'city_id_C042', 'city_id_C043', 'city_id_C044', 'city_id_C045', 'city_id_C046', 'city_id_C047', 'city_id_C048', 'city_id_C049', 'city_id_C051', 'city_id_C053', 'city_id_C054', 'city_id_C055', 'city_id_C056', 'city_id_C057', 'city_id_C058', 'city_id_C059', 'city_id_C061', 'city_id_C062', 'city_id_C064', 'city_id_C065', 'city_id_C066', 'city_id_C067', 'city_id_C068', 'city_id_C069', 'city_id_C070', 'city_id_C071', 'city_id_C072', 'city_i

In [242]:
new_features_order = ['min_temp_c', 'max_temp_c'] + [col for col in features_new if col not in ['min_temp_c', 'max_temp_c']]
print(new_features_order)

['min_temp_c', 'max_temp_c', 'city_id_C001', 'city_id_C002', 'city_id_C003', 'city_id_C004', 'city_id_C005', 'city_id_C007', 'city_id_C008', 'city_id_C009', 'city_id_C010', 'city_id_C011', 'city_id_C012', 'city_id_C013', 'city_id_C014', 'city_id_C015', 'city_id_C016', 'city_id_C017', 'city_id_C018', 'city_id_C020', 'city_id_C022', 'city_id_C023', 'city_id_C024', 'city_id_C025', 'city_id_C027', 'city_id_C028', 'city_id_C029', 'city_id_C030', 'city_id_C031', 'city_id_C033', 'city_id_C034', 'city_id_C035', 'city_id_C036', 'city_id_C037', 'city_id_C038', 'city_id_C039', 'city_id_C040', 'city_id_C042', 'city_id_C043', 'city_id_C044', 'city_id_C045', 'city_id_C046', 'city_id_C047', 'city_id_C048', 'city_id_C049', 'city_id_C051', 'city_id_C053', 'city_id_C054', 'city_id_C055', 'city_id_C056', 'city_id_C057', 'city_id_C058', 'city_id_C059', 'city_id_C061', 'city_id_C062', 'city_id_C064', 'city_id_C065', 'city_id_C066', 'city_id_C067', 'city_id_C068', 'city_id_C069', 'city_id_C070', 'city_id_C0

In [243]:
X_submission = submission_data[new_features_order]
submission_data = submission_data.copy()
submission_data['avg_temp_c'] = model_avg.predict(X_submission)

In [244]:
submission = submission_data[['submission_ID', 'avg_temp_c']]
submission.to_csv('submission3.csv', index=False)